队列负责“顺序与缓冲”，业务模块负责“怎么处理任务”。
底层实现：分段双向链表
先想“数据怎么流动”，再选数据结构。

deque vs queue.Queue？


In [ ]:
from collections import deque
import time
import random

task_queue = deque()
# 入队（enqueue）
# 创建10个png
pngs = ["a.png", "b.png", "c.png", "d.png", "e.png", "f.png"]
[task_queue.append(f"downlaod: {png}") for png in pngs]

# 出队（dequeue）：从左边拿，保证先进先出
while task_queue:
    # list 删除第 0 个元素会导致后面所有元素整体左移
    # current_task = task_queue.pop(0)  # ❌ 看起来很对，其实会慢
    current_task = task_queue.popleft()
    time.sleep(random.uniform(0.5, 1))
    print(f"processing -> {current_task}")


In [ ]:
from collections import deque

TASK = ["A", "B", "C"]


def print_task(tasks: deque[str]):
    while tasks:
        task = tasks.popleft()
        if task is None:
            print("任务结束")
            break
        # 模拟任务时间
        time.sleep(random.uniform(0.5, 1))
        print(f"printed:{task}")


def run():
    # 创建队列
    q = deque()
    [q.append(task) for task in TASK]
    q.append(None)
    print_task(q)


run()


In [ ]:
from collections import deque
from typing import Iterable

TASK = ["A", "B", "C"]


def run_print_jobs(jobs: Iterable[str]):
    jobs_queue: deque[str] = deque(jobs)
    results: list[str] = []

    while jobs_queue:
        job = jobs_queue.popleft()
        print(f"开始处理任务：{job}")
        results.append(f"printed:{job}")
    return results


run_print_jobs(TASK)


In [ ]:
from queue import Queue
from threading import Thread
import time
import random


def producer(q: Queue, name):
    for i in range(10):
        task = f"task-{i}"
        q.put(task)
        print(f"生产者{name} 创建任务：{task} OK")
        time.sleep(random.uniform(0, 1))
    q.put(None)
    print("任务创建完成。")


def consumer(q: Queue, name):
    while True:
        task = q.get()
        if task is None:
            print("队列为空，结束任务")
            break
        time.sleep(random.uniform(0, 1))
        q.task_done()
        print(f"消费者{name} 处理任务：{task} OK")

    print("所有任务处理完毕。")


def main():
    q = Queue()
    t1 = Thread(
        target=producer,
        args=(q, "t1"),
    )
    t2 = Thread(
        target=consumer,
        args=(q, "t2"),
    )
    t1.start()
    t2.start()
    t1.join()
    t2.join()


main()


In [ ]:
# 练习
from queue import Queue
import threading
import random
import time


def producer(task_queue: Queue, task_data: list[str], name: str):
    for data in task_data:
        task_queue.put(data)
        print(f"{name}生产者:{data} 创建成功 OK")
    task_queue.put(None)
    print(f"{name}生产者: 创建完成")
    return task_queue


result: list[str] = []


def consumer(q: Queue, result: list[str], name: str):
    while True:
        try:
            task = q.get()
            if task is None:
                return
            result.append(f"done:{task}")
            print(f"{name}消费者:{task} 处理完成")
        finally:
            q.task_done()


def run_workers(task_data: list[str]):
    # 注意：
    # 关键点：Python 函数传参传的是“对象引用”（更准确：把引用绑定到形参名上）。
    task_queue = Queue()
    producer_params = dict(task_queue=task_queue, task_data=task_data, name="t1")
    t1 = threading.Thread(target=producer, kwargs=producer_params)

    result: list[str] = []
    consumer_params = dict(q=task_queue, result=result, name="t2")
    t2 = threading.Thread(target=consumer, kwargs=consumer_params)
    t1.start()
    t2.start()

    t1.join()
    t2.join()
    assert result == ["done:A", "done:B", "done:C"]


task_data = ["A", "B", "C"]
run_workers(task_data)


In [ ]:
# 状态保存
def mutate(xs: list[int]) -> None:
    xs.append(1)


"""
obj = {
    args:{xs}
    local:{xs:[99]}
}
"""


def rebind(xs: list[int]) -> None:
    xs = [999]
    xs.append(1)


data = []
mutate(data)
assert data == [1]

data = []
rebind(data)
assert data == []  # 为什么？


def fn():
    xs = [99]
    return xs


fn.xs = "hello"
print(fn())
print(fn.xs)

print(fn.__dict__)


In [19]:
def fnA(data: list[str]):
    data: list[int] = [10]
    print(data)
    print("函数内局域变量：", locals())


fnA.attrs = ["A", "B"]
# python把函数的参数保存在函数对象的属性中。
fnA.data = "hello world"
print("函数对象上的属性：", fnA.__dict__)
fnA(["a", "b"])


函数对象上的属性： {'attrs': ['A', 'B'], 'data': 'hello world'}
[10]
函数内局域变量： {'data': [10]}
